# 공행성(선후행 관계) 쌍 탐색 및 예측 모델

## 목표
시차별 상관관계 분석을 기반으로 공행성 쌍을 도출하고, RandomForest와 GradientBoosting 앙상블 모델을 사용하여 2025년 8월 무역량을 예측합니다.

## 분석 전략
1. **시차별 상관관계 분석**: Cross-correlation을 통한 최적 시차 탐색
2. **공행성 쌍 탐색**: 피어슨 상관계수 기반으로 선후행 관계 도출
3. **Feature Engineering**: 시계열 특성 및 공행성 특성 추출
4. **앙상블 모델 학습**: RandomForest + GradientBoosting
5. **추론 및 제출**: 2025년 8월 무역량 예측


## 1. 라이브러리 임포트


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from tqdm import tqdm
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 로드 완료")


## 2. 데이터 로드 및 전처리


In [ ]:
# 데이터 로드
train = pd.read_csv('../data/raw/train.csv')
print(f"원본 데이터 shape: {train.shape}")
print(f"기간: {train['year'].min()}-{train['month'].min():02d} ~ {train['year'].max()}-{train['month'].max():02d}")
print(f"고유 item_id 수: {train['item_id'].nunique()}")

# year, month, item_id 기준으로 value 합산 (seq만 다르다면 value 합산)
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

print(f"\n피벗 테이블 shape: {pivot.shape}")
print(f"월별 데이터 기간: {pivot.columns.min()} ~ {pivot.columns.max()}")
print(f"총 개월 수: {len(pivot.columns)}")

# 기본 통계
print(f"\n각 item_id별 통계:")
print(f"  - 평균 거래량: {pivot.values[pivot.values > 0].mean():.2f}")
print(f"  - 중위수 거래량: {np.median(pivot.values[pivot.values > 0]):.2f}")
print(f"  - 0이 아닌 값의 비율: {(pivot.values > 0).sum() / pivot.size * 100:.2f}%")


## 3. 공행성 쌍 탐색

### 가설
- **가설 1**: 특정 item A의 변동이 k개월 후 item B의 변동에 영향을 미친다면, A와 B는 공행성 쌍이다.
- **검증 방법**: Cross-correlation을 통해 최적 시차(lag) 탐색
- **기대 결과**: 특정 lag에서 유의미한 상관계수 발견


In [ ]:
def safe_corr(x, y):
    """
    안전한 상관계수 계산 함수
    분산이 0인 경우 0.0 반환
    """
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(
    pivot, 
    max_lag=6, 
    min_nonzero=12, 
    corr_threshold=0.4
):
    """
    공행성 쌍을 탐색하는 함수
    
    Parameters
    ----------
    pivot : pd.DataFrame
        item_id별 월별 거래량 매트릭스 (index: item_id, columns: ym)
    max_lag : int, default=6
        리더와 팔로워 품목 간 최대 탐색 시차(개월)
        - lag=1이면 리더의 변동이 1개월 뒤 팔로워에 영향을 주는지 확인
        - lag=2,3,...,k까지 값을 늘릴수록 더 오래된 영향/지연 효과 패턴도 포함
    min_nonzero : int, default=12
        0이 아닌 거래량을 기록한 최소 월 개수
        - 너무 작으면 빈약한 시계열이 노이즈를 많이 줄 수 있음
        - 너무 크면 데이터가 부족해져 유의미한 쌍을 찾지 못할 수 있음
        - 월 단위라면 최소 1년(12개월) 정도를 기본으로 사용
    corr_threshold : float, default=0.4
        피어슨 상관계수의 임계값(절댓값 기준)
        - 이 값보다 크거나 같은 경우만 공행성쌍으로 간주
        - 0.3~0.4 이상이면 실무적으로 '상당한 경향성'이 있다고 인정
        - 0.4~0.6: 신뢰성과 검출 건수의 균형 (실무에서 가장 추천)
    
    Returns
    -------
    pd.DataFrame
        공행성 쌍 결과 (leading_item_id, following_item_id, best_lag, max_corr)
    """
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)

    results = []

    for i, leader in tqdm(enumerate(items), desc="공행성쌍 탐색"):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0

            # lag = 1 ~ max_lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue
                corr = safe_corr(x[:-lag], y[lag:])
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            # 임계값 이상이면 공행성쌍으로 채택
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })

    pairs = pd.DataFrame(results)
    return pairs

# 파라미터 설정
MAX_LAG = 6
MIN_NONZERO = 12
CORR_THRESHOLD = 0.4

print(f"파라미터 설정:")
print(f"  - MAX_LAG: {MAX_LAG}")
print(f"  - MIN_NONZERO: {MIN_NONZERO}")
print(f"  - CORR_THRESHOLD: {CORR_THRESHOLD}")

# 공행성 쌍 탐색 실행
pairs = find_comovement_pairs(
    pivot,
    max_lag=MAX_LAG,
    min_nonzero=MIN_NONZERO,
    corr_threshold=CORR_THRESHOLD
)

print(f"\n탐색된 공행성 쌍 수: {len(pairs)}")
if len(pairs) > 0:
    print(f"\n상위 10개 쌍:")
    display(pairs.head(10))


## 4. Feature Engineering 및 학습 데이터 생성


In [ ]:
def build_training_data_enhanced(pivot, pairs, use_log_transform=False):
    """
    향상된 Feature Engineering을 적용한 학습 데이터 생성
    
    Input Features:
      - b_t: 현재 달 팔로워 품목의 총 무역량
      - b_t_1: 직전 달 팔로워 품목의 총 무역량
      - a_t_lag: lag 반영된 리더 품목의 총 무역량
      - max_corr: 최대 상관계수
      - best_lag: 최적 시차
      - b_t_ma3: 팔로워 품목의 3개월 이동평균
      - a_t_lag_ma3: 리더 품목의 3개월 이동평균
      - b_trend: 팔로워 품목의 변화율
      - a_trend: 리더 품목의 변화율
      - b_t_scaled: 정규화된 팔로워 품목 값
      - a_t_lag_scaled: 정규화된 리더 품목 값
      - a_t_lag_weighted: 상관계수 가중치가 적용된 리더 품목 값
      - lag_1, lag_2, lag_3plus: lag별 특성
    
    Target:
      - b_t_plus_1: 다음 달 팔로워 품목의 총 무역량
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t+1이 존재하고, t-lag >= 0인 구간만 학습에 사용
        for t in range(max(lag, 2), n_months - 1):  # 최소 2개월 전 데이터 필요
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            b_t_2 = b_series[t - 2] if t >= 2 else 0.0
            a_t_lag = a_series[t - lag]
            a_t_lag_1 = a_series[t - lag - 1] if t - lag - 1 >= 0 else 0.0
            b_t_plus_1 = b_series[t + 1]

            # 기본 feature
            features = {
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
            }
            
            # 추가 feature: 이동평균
            window = min(3, t + 1)
            features["b_t_ma3"] = np.mean(b_series[max(0, t - window + 1):t + 1])
            features["a_t_lag_ma3"] = np.mean(a_series[max(0, t - lag - window + 1):t - lag + 1]) if t - lag >= 0 else 0.0
            
            # 추가 feature: 트렌드 (변화율)
            features["b_trend"] = (b_t - b_t_1) / (b_t_1 + 1e-6) if b_t_1 > 0 else 0.0
            features["b_trend_2"] = (b_t_1 - b_t_2) / (b_t_2 + 1e-6) if b_t_2 > 0 else 0.0
            features["a_trend"] = (a_t_lag - a_t_lag_1) / (a_t_lag_1 + 1e-6) if a_t_lag_1 > 0 else 0.0
            
            # 추가 feature: 정규화된 값
            b_mean = np.mean(b_series[:t+1])
            a_mean = np.mean(a_series[:t-lag+1]) if t - lag >= 0 else 1.0
            features["b_t_scaled"] = b_t / (b_mean + 1e-6)
            features["a_t_lag_scaled"] = a_t_lag / (a_mean + 1e-6)
            
            # 추가 feature: 상관관계 가중치
            features["a_t_lag_weighted"] = a_t_lag * abs(corr)
            
            # 추가 feature: lag별 특성
            features["lag_1"] = 1.0 if lag == 1 else 0.0
            features["lag_2"] = 1.0 if lag == 2 else 0.0
            features["lag_3plus"] = 1.0 if lag >= 3 else 0.0
            
            # Target
            if use_log_transform:
                features["target"] = np.log1p(b_t_plus_1)
            else:
                features["target"] = b_t_plus_1

            rows.append(features)

    df_train = pd.DataFrame(rows)
    return df_train

# 학습 데이터 생성
USE_LOG_TRANSFORM = False
df_train_model = build_training_data_enhanced(pivot, pairs, use_log_transform=USE_LOG_TRANSFORM)
print(f'생성된 학습 데이터의 shape: {df_train_model.shape}')
print(f'Feature 목록: {[col for col in df_train_model.columns if col != "target"]}')


## 5. 모델 학습 (최적 하이퍼파라미터 적용)

README.md의 최적 하이퍼파라미터를 적용하여 RandomForest와 GradientBoosting 모델을 학습합니다.


In [ ]:
# Feature 선택
feature_cols = [col for col in df_train_model.columns if col != 'target']
train_X = df_train_model[feature_cols].values
train_y = df_train_model["target"].values

print(f"Feature 개수: {len(feature_cols)}")
print(f"학습 샘플 수: {len(train_X)}")

# RandomForest 최적 하이퍼파라미터 (README.md 기준)
rf_params = {
    'n_estimators': 50,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': None,
    'max_depth': 25,
    'random_state': 42,
    'n_jobs': -1
}

# GradientBoosting 최적 하이퍼파라미터 (README.md 기준)
gb_params = {
    'n_estimators': 300,
    'min_samples_split': 2,
    'min_samples_leaf': 2,
    'max_depth': 10,
    'learning_rate': 0.15,
    'subsample': 1.0,
    'random_state': 42
}

print("\n=== RandomForest 하이퍼파라미터 ===")
for param, value in rf_params.items():
    print(f"  {param}: {value}")

print("\n=== GradientBoosting 하이퍼파라미터 ===")
for param, value in gb_params.items():
    print(f"  {param}: {value}")

# 모델 생성 및 학습
print("\n=== 모델 학습 시작 ===")
rf_model = RandomForestRegressor(**rf_params)
rf_model.fit(train_X, train_y)
print("RandomForest 학습 완료")

gb_model = GradientBoostingRegressor(**gb_params)
gb_model.fit(train_X, train_y)
print("GradientBoosting 학습 완료")


## 6. 앙상블 예측 함수 정의


In [ ]:
def predict_ensemble(pivot, pairs_df, models, weights, feature_cols, target_date, use_log_transform=False):
    """
    앙상블 모델을 사용한 예측 함수
    
    Parameters:
    -----------
    pivot : pd.DataFrame
        전체 피벗 테이블
    pairs_df : pd.DataFrame
        공행성 쌍 데이터프레임
    models : dict
        모델 딕셔너리 (예: {'rf': RandomForest 모델, 'gb': GradientBoosting 모델})
    weights : dict
        가중치 딕셔너리 (예: {'rf': 0.2, 'gb': 0.8})
    feature_cols : list
        Feature 컬럼 리스트
    target_date : datetime
        예측 대상 날짜 (이 날짜 이전의 데이터만 사용)
    use_log_transform : bool
        로그 변환 사용 여부
    
    Returns:
    --------
    pd.DataFrame
        예측 결과 (leading_item_id, following_item_id, value)
    """
    # target_date 이전까지의 데이터만 사용
    pivot_for_pred = pivot.loc[:, pivot.columns < target_date].copy()
    
    if len(pivot_for_pred.columns) == 0:
        raise ValueError(f"target_date {target_date} 이전의 데이터가 없습니다.")
    
    months = pivot_for_pred.columns.to_list()
    n_months = len(months)
    
    # 가장 마지막 달 index
    t_last = n_months - 1
    t_prev = t_last - 1 if t_last > 0 else t_last
    t_prev2 = t_last - 2 if t_last >= 2 else 0

    preds = []

    for row in tqdm(pairs_df.itertuples(index=False), desc="예측 중"):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot_for_pred.index or follower not in pivot_for_pred.index:
            continue

        a_series = pivot_for_pred.loc[leader].values.astype(float)
        b_series = pivot_for_pred.loc[follower].values.astype(float)

        # t_last - lag 가 0 이상인 경우만 예측
        if t_last - lag < 0:
            continue

        # Feature 생성 (학습 시와 동일한 방식)
        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev] if t_prev >= 0 else 0.0
        b_t_2 = b_series[t_prev2] if t_prev2 >= 0 else 0.0
        a_t_lag = a_series[t_last - lag]
        a_t_lag_1 = a_series[t_last - lag - 1] if t_last - lag - 1 >= 0 else 0.0

        # Feature 벡터 생성
        features = {
            "b_t": b_t,
            "b_t_1": b_t_1,
            "a_t_lag": a_t_lag,
            "max_corr": corr,
            "best_lag": float(lag),
            "b_t_ma3": np.mean(b_series[max(0, t_last - 2):t_last + 1]),
            "a_t_lag_ma3": np.mean(a_series[max(0, t_last - lag - 2):t_last - lag + 1]) if t_last - lag >= 0 else 0.0,
            "b_trend": (b_t - b_t_1) / (b_t_1 + 1e-6) if b_t_1 > 0 else 0.0,
            "b_trend_2": (b_t_1 - b_t_2) / (b_t_2 + 1e-6) if b_t_2 > 0 else 0.0,
            "a_trend": (a_t_lag - a_t_lag_1) / (a_t_lag_1 + 1e-6) if a_t_lag_1 > 0 else 0.0,
            "b_t_scaled": b_t / (np.mean(b_series[:t_last+1]) + 1e-6),
            "a_t_lag_scaled": a_t_lag / (np.mean(a_series[:t_last-lag+1]) + 1e-6) if t_last - lag >= 0 else 0.0,
            "a_t_lag_weighted": a_t_lag * abs(corr),
            "lag_1": 1.0 if lag == 1 else 0.0,
            "lag_2": 1.0 if lag == 2 else 0.0,
            "lag_3plus": 1.0 if lag >= 3 else 0.0,
        }
        
        X_test = np.array([[features[col] for col in feature_cols]])

        # 각 모델의 예측값 계산
        ensemble_pred = 0.0
        for model_name, model in models.items():
            y_pred_single = model.predict(X_test)[0]
            
            # 로그 변환 사용 시 역변환
            if use_log_transform:
                y_pred_single = np.expm1(y_pred_single)
            
            # 가중 평균
            ensemble_pred += weights[model_name] * y_pred_single

        y_pred = ensemble_pred

        # 후처리: 음수 방지 및 정수 변환
        y_pred = max(0.0, float(y_pred))
        
        # 추가 후처리: 이상치 제한
        if b_t > 0:
            # 현재 값의 20배를 넘지 않도록 제한
            y_pred = min(y_pred, b_t * 20)
            # 최근 트렌드 반영 (선택적)
            if b_t_1 > 0:
                trend = b_t / b_t_1
                y_pred = y_pred * (0.7 + 0.3 * min(trend, 2.0))  # 트렌드 반영하되 과도하지 않게
        
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })

    df_pred = pd.DataFrame(preds)
    return df_pred

print("앙상블 예측 함수 정의 완료")


## 7. 추론 실행 및 제출 파일 생성

train.csv의 모든 데이터를 활용하여 2025년 8월 데이터를 예측합니다.


In [ ]:
# 추론 대상: 2025년 8월
target_date = pd.to_datetime("2025-08-01")

# 앙상블 모델 및 가중치 설정 (README.md 기준)
models_dict = {
    'rf': rf_model,
    'gb': gb_model
}

# 최적 가중치 (README.md 기준: RF 0.20, GB 0.80)
best_weights = {
    'rf': 0.20,
    'gb': 0.80
}

print("=== 앙상블 예측 시작 ===")
print(f"RF 가중치: {best_weights['rf']:.2f}")
print(f"GB 가중치: {best_weights['gb']:.2f}")
print(f"추론 대상 날짜: {target_date}\n")

# 추론 실행 (2025-08-01 예측)
submission = predict_ensemble(
    pivot, 
    pairs, 
    models_dict, 
    best_weights, 
    feature_cols, 
    target_date,
    use_log_transform=USE_LOG_TRANSFORM
)

print(f"\n생성된 예측 결과 수: {len(submission)}")
print(f"예측값 통계:")
print(submission['value'].describe())

# sample_submission.csv 형태로 변환
# sample_submission.csv의 모든 쌍을 포함하도록 확장
sample_submission = pd.read_csv('../data/raw/sample_submission.csv')

# 예측 결과를 딕셔너리로 변환
pred_dict = dict(zip(
    zip(submission['leading_item_id'], submission['following_item_id']),
    submission['value']
))

# sample_submission의 모든 쌍에 대해 예측값 할당
final_submission = sample_submission.copy()
final_submission['value'] = final_submission.apply(
    lambda row: pred_dict.get((row['leading_item_id'], row['following_item_id']), 0),
    axis=1
)

print(f"\n최종 제출 파일 shape: {final_submission.shape}")
print(f"예측값이 있는 쌍 수: {(final_submission['value'] > 0).sum()}")
print(f"예측값 합계: {final_submission['value'].sum():,.0f}")

# experiments 폴더에 저장
os.makedirs('../experiments', exist_ok=True)

# 파일명 생성 (타임스탬프 포함)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'ensemble_rf_gb_{timestamp}.csv'
output_path = f'../experiments/{output_filename}'

final_submission.to_csv(output_path, index=False)
print(f"\n추론 결과가 {output_path}에 저장되었습니다.")
